<center>به نام خدا</center>
<br>
<center> درس بازیابی پیشرفته اطلاعات - بهار ۱۴۰۰ - ۱۴۰۱ </center>
<br>
<center>تمرین سوم</center>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# address = "MIRNews1401"
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(address))
# %cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/MIRNews1401


In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.0 MB/s 
     |████████████████████████████████| 233 kB 55.7 MB/s 
     |████████████████████████████████| 1.4 MB 45.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=21d3efbfd2a5400c285ade46bddbe6d21def4baa947e0b219aab4b9772e5bf21
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154128 sha256=ada42d676cdf9b64de8763309fb83107d2483135e13461039826c96be54c0d76
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
import json
import math
from tqdm import tqdm
from hazm import *
import numpy as np
import pandas as pd
from collections import Counter
import numexpr as ne

# 1 preprocessing

In [ ]:
# dictData = {}
# with open("news_all_text.json", encoding="utf8") as d:
#     dictData = json.load(d)

In [ ]:
# dictData[0].keys()

dict_keys(['@timestamp', 'categories', 'id', 'title', 'content', 'url', 'tags'])

In [ ]:
stop_words = stopwords_list()
normalizer = Normalizer()
lemmatizer = Lemmatizer()

def preprocess_text(document):
        document = normalizer.normalize(document)
        
        tokens = word_tokenize(document)
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in stop_words]
        tokens = [word for word in tokens if len(word) > 2]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

In [ ]:
# news_list = []
# news_details = []

In [ ]:
# news_list.clear()
# news_details.clear()
# for new in tqdm(dictData):
#     changed_text = preprocess_text(new['content'])
#     news_list.append(changed_text)
#     news_details.append({'url':new['url'], 'id': new['id'], 'title': new['title']})

100%|██████████| 5000/5000 [00:18<00:00, 275.93it/s]


In [ ]:
# len(news_list)

5000

In [ ]:
# np.save("./data/news_details", news_details)

In [ ]:
news_details = np.load("./data/news_details.npy", allow_pickle=True)

# 2. Retrieval 

<div dir=rtl>
در بخش‌های زیر، به شکل مجزا، یک سیستم بازیابی را پیاده‌سازی می‌کنیم.
</div>

# 2.1 Boolean Retrieval

<div dir=rtl>
در این قسمت بازیابی بولین را انجام می‌دهیم. البته برای کسب نتیجه بهتر، از فرکانس نیز استفاده کرده‌ایم و در زمان‌های برابری، معیار فرکانس را نیز در نظر گرفتیم.
</div>

In [ ]:
'''
list_all_unique_tokens()
This function gets a list of rm strings as input and outputs a list of unique 
tokens extracted from them.
input: a list of rm strings
output:
     1. a list of unique tokens (in a set) in each document
     2. a list of all unique tokens
'''


def list_all_unique_tokens(rm_strings_list):

    # tokenize every string 
    list_of_lists = [word_tokenize(rm_string) for rm_string in tqdm(rm_strings_list)]

    # convert every list to a set (remove duplicate values)
    list_of_sets = [set(item) for item in tqdm(list_of_lists)]

    # full union (set of all unique tokens)
    unique_list = list(set.union(*list_of_sets))

    # return
    return list_of_lists, list_of_sets, unique_list


In [ ]:
# list_of_token_lists, list_of_token_sets, unique_token_list = list_all_unique_tokens(news_list)

100%|██████████| 5000/5000 [00:00<00:00, 44497.37it/s]


<div dir=rtl>
در بخش زیر ماتریس‌های فرکانس و بولین ایجاد شده‌اند. همان Term-doc matrix
</div>

In [ ]:
# # Initialize Frequency Matrix
# frequency_matrix = np.empty([len(unique_token_list), len(news_list)])
# frequency_matrix.fill(0)

# # Fill Frequency matrix
# for col in tqdm(range(np.shape(frequency_matrix)[1])):
#     occurence_counter = Counter(list_of_token_lists[col])
#     for row in range(np.shape(frequency_matrix)[0]):
#         frequency_matrix[row, col] = occurence_counter[unique_token_list[row]]

100%|██████████| 5000/5000 [03:10<00:00, 26.25it/s]


In [ ]:
# np.save("./data/boolean_frequency_matrix", frequency_matrix)
# np.save("./data/boolean_unique_token_list", unique_token_list)

In [ ]:
frequency_matrix = np.load("./data/boolean_frequency_matrix.npy")
unique_token_list = np.load("./data/boolean_unique_token_list.npy")

In [ ]:
# Convert to DataFrame
frequency_matrix_df = pd.DataFrame(data = frequency_matrix, 
                                   index = unique_token_list, 
                                   columns = list(range(frequency_matrix.shape[1])))

boolean_matrix = frequency_matrix.copy()
boolean_matrix[boolean_matrix > 0] = 1

# Convert to DataFrame
boolean_matrix_df = pd.DataFrame(data = boolean_matrix, 
                                 index = unique_token_list, 
                                 columns = list(range(frequency_matrix.shape[1])))

In [ ]:
# Convert to DataFrame
frequency_matrix_df = pd.DataFrame(data = frequency_matrix, 
                                   index = unique_token_list, 
                                   columns = list(range(frequency_matrix.shape[1])))

boolean_matrix = frequency_matrix.copy()
boolean_matrix[boolean_matrix > 0] = 1

# Convert to DataFrame
boolean_matrix_df = pd.DataFrame(data = boolean_matrix, 
                                 index = unique_token_list, 
                                 columns = list(range(frequency_matrix.shape[1])))

In [ ]:
# boolean_matrix_df.shape

(29697, 5000)

<div dir=rtl>
قسمت اصلی این بخش، تابع زیر است. که با دریافت ماتریس‌های بولین و فرکانس، نزدیک‌ترین نتایج یک کوئری را برگشت می‌هد.
</div>

In [ ]:

'''
boolean_retrieval()
This function takes a query as input and returns a list of indices of documents as output
A Query consists of two parts:
    1. A simple string of words
    2. A list of words not to be included in the output documents (This part is optional)
'''


def boolean_retrieval(boolean_matrix_df, frequency_matrix_df, k, query_string, not_raw_tokens=[]):

    # get all query tokens
    query_raw_tokens = word_tokenize(query_string)

    # remove stop words
    query_tokens = [lemmatizer.lemmatize(token) for token in query_raw_tokens if token not in stop_words]

    # remove stop words from not_tokens
    not_tokens = [lemmatizer.lemmatize(token) for token in not_raw_tokens if token not in stop_words]

    # get all unique tokens which are already in our readmes 
    unique_tokens = boolean_matrix_df.index.values.tolist()

    # available tokens in query
    available_query_tokens = [token for token in query_tokens if token in unique_tokens]

    # only choose specific rows of df which correspond to query tokens
    summary_df = boolean_matrix_df.loc[available_query_tokens]
    summary_frequency_df = frequency_matrix_df.loc[available_query_tokens]

    # available tokens in not_tokens
    available_not_tokens = [token for token in not_tokens if token in unique_tokens]

    # df consisting of only available_not_tokens
    summary_not_df = boolean_matrix_df.loc[available_not_tokens]
    arr = summary_not_df.values
    summary_not_df = pd.DataFrame(ne.evaluate('1 - arr'), columns=summary_not_df.columns, index=summary_not_df.index)
    summary_not_frequency_df = frequency_matrix_df.loc[available_not_tokens]
    arr = summary_not_frequency_df.values
    summary_not_frequency_df = pd.DataFrame(ne.evaluate('0 - arr'), columns=summary_not_frequency_df.columns, index=summary_not_df.index)

    # specify boolean rating
    rating_1 = summary_df.sum().tolist()
    rating_2 = summary_not_df.sum().tolist()
    boolean_rating = [x + y for x, y in zip(rating_1, rating_2)]

    # specify frequency rating
    rating_1 = summary_frequency_df.sum().tolist()
    rating_2 = summary_not_frequency_df.sum().tolist()
    frequency_rating = [x + y for x, y in zip(rating_1, rating_2)]

    # sort indices based on ratings
    raw_indices = list(range(len(boolean_matrix_df.columns)))
    list_of_tuples = [(bool_item, freq_item) for bool_item, freq_item in zip(boolean_rating, frequency_rating)]
    zipped = zip(list_of_tuples, raw_indices)
    sorted_indices = [item for _, item in sorted(zipped, key=lambda pair: pair[0], reverse=True)]
    
    return sorted_indices[:k]

# 2.2 transformer

<div dir=rtl>
در این بخش با استفاده از ترنسفورمرها بازیابی را انجام داده‌ایم. به طور خاص از sentence transformer
استفاده کرده‌ایم. توجه کنید که این یک مدل trainشده است.
</div>

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 4.4 MB 32.4 MB/s 
     |████████████████████████████████| 1.2 MB 65.9 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a59ac0d1958a2f0a316f162825dcbbd7ed98876b1b2dec33f96d9fa313324daf
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import util, SentenceTransformer

In [ ]:
transformer_model = SentenceTransformer('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/519 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/651M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/m3hrdadfi_bert-fa-base-uncased-wikinli-mean-tokens. Creating a new one with MEAN pooling.


<div dir=rtl>
در بلاک زیر، با استفاده از مدل دریافت شده، لیست فایل‌ها را کدگذاری می‌کنیم. مشابه حالت قبل نتیجه را برای استفاه‌های بعدی ذخیره می‌کنیم.
</div>

In [ ]:
# transformer_doc_embeddings = transformer_model.encode(news_list, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
# np.save("./data/transformer_doc_embeddings", transformer_doc_embeddings)

In [ ]:
transformer_doc_embeddings = np.load("./data/transformer_doc_embeddings.npy")

In [ ]:
# transformer_doc_embeddings.shape

(5000, 768)

# 2.3 Tf-Idf

<div dir=rtl>
در اینجا سراغ tf-idf
می‌رویم. توجه کنید که بخشی از داده‌ها در روش بعد نیز به کار می‌روند.
</div>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<div dir=rtl>
ابتدا یک مدل tf-idf
ساخته‌ایم و سپس لیست فایل‌ها را بر روی آن فیت کرده‌ایم. خود فایل‌ها را نیز کدگذاری کرده‌ایم.
</div>

In [ ]:
tfidf_vectorizer = TfidfVectorizer(smooth_idf=False, norm="l2")
# tfidf_embeddings = tfidf_vectorizer.fit_transform(news_list)

In [ ]:
# json.dump(tfidf_vectorizer.vocabulary_, open('./data/tfidf_vocabulary.json', mode = 'w'))
# np.save("./data/tfidf_idf", tfidf_vectorizer.idf_)

In [ ]:
tfidf_vocabulary = json.load(open('./data/tfidf_vocabulary.json', mode = 'r'))
tfidf_idf = np.load("./data/tfidf_idf.npy")

In [ ]:
tfidf_vectorizer.idf_ = tfidf_idf
tfidf_vectorizer.vocabulary_ = tfidf_vocabulary

In [ ]:
from scipy import sparse

In [ ]:
# sparse.save_npz("./data/tfidf_embeddings", tfidf_embeddings)

In [ ]:
tfidf_embeddings = sparse.load_npz("./data/tfidf_embeddings.npz")

# 2.4 fasttext

<div dir=rtl>
آخرین روش، بردار وزن‌دار tf-idf بر روی fasttext است.
<br>
به این صورت که ابتدا از روی مجموعه فایل‌ها، هر کدام از لغات یک بردار با طول مشخص دریافت می‌کنند. (به روش fasttext) سپس با استفاده از idf
کلمات، هرکدام از فایل‌ها را به شکل یک مجموع وزن‌دار از بردارهای اولیه در می‌آوریم.
</div>

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
# word_tokenized_corpus = [word_tokenize(sent) for sent in news_list]

<div dir=rtl>
در قسمت زیر مدل از روی لغات فایل‌ها آموزش داده می‌شود.
</div>

In [ ]:
# ft_model = FastText(word_tokenized_corpus)

In [ ]:
# ft_model.save("./data/fasttext.model")

In [ ]:
ft_model = FastText.load("./data/fasttext.model")

<div dir=rtl>
در اینجا لیست لغات و وزن idf آن‌ها را از مدل tf-idf دریافت می‌کنیم.
</div>

In [ ]:
features = tfidf_vectorizer.get_feature_names_out()
idfs = tfidf_vectorizer.idf_
embedding_size = 100

<div dir=rtl>
و در نهایت در قسمت زیر که خیلی طول می‌کشد، هر فایل از روی مجموعه لغات آن و کد آن‌ها، کدگذاری می‌شود. همان مجموع وزن‌دار fasttext
</div>

In [ ]:
# fast_weighted_embeddings = []
# docs_size = len(news_list)
# absent_words = 0
# for i in tqdm(range(docs_size)):
#     init_emb = np.zeros(embedding_size)
#     for word in word_tokenize(news_list[i]):
#         try:
#             emb = ft_model.wv[word]
#             resault = np.where(features == word)[0]
#             if resault.size > 0:
#                 init_emb += idfs[resault[0]] * emb / np.linalg.norm(emb)
#         except Exception as e:
#             absent_words += 1
#     fast_weighted_embeddings.append(init_emb)
# print('number of absent words from model:', absent_words)

100%|██████████| 5000/5000 [12:38<00:00,  6.59it/s]

number of absent words from model: 449


In [ ]:
# np.save("./data/fast_weighted_embeddings", fast_weighted_embeddings)

In [ ]:
fast_weighted_embeddings = np.load("./data/fast_weighted_embeddings.npy")

In [ ]:
# np.array(fast_weighted_embeddings).shape

(5000, 100)

<div dir=rtl>
در بخش جستجو مشابه فایل‌ها، باید لغات کوئری را کدگذاری کنیم و سپس از روی idf لغات، آن‌ها در ترکیب کنیم.
و از روی فاصله کسینوسی، فایل‌های نزدیک به کوئری را پیدا کنیم.
</div>

# 3. query

In [ ]:
def query_expansion_embedding(query_emb, rel_list, non_rel_list):
    a = 1
    b = 0.8
    c = 0.1
    resault = a * query_emb + b * np.sum(rel_list)/len(rel_list) - c * np.sum(non_rel_list)/len(non_rel_list)
    return resault                                                                              

In [ ]:
def boolean_search(text, K):
    my_indices = boolean_retrieval(boolean_matrix_df, frequency_matrix_df, K, text)
    return my_indices

In [ ]:
def transformer_search(text, K, expansion=False):
    query = preprocess_text(text)
    transformer_query_embedding = transformer_model.encode(query)
    transformer_list_sim = util.cos_sim(transformer_query_embedding, transformer_doc_embeddings)
    if expansion:
        sorted_list_sim = np.argsort(np.array(transformer_list_sim[0]))
        most_similar_emb = [transformer_doc_embeddings[i] for i in sorted_list_sim[::-1][:10]]
        least_similar_emb = [transformer_doc_embeddings[i] for i in sorted_list_sim[:][:10]]
        expanded = query_expansion_embedding(transformer_query_embedding, most_similar_emb, least_similar_emb)
        transformer_list_sim = util.cos_sim(expanded, transformer_doc_embeddings)
    
    transformer_most_similar_doc_indices = np.argsort(np.array(transformer_list_sim[0]))[::-1][:K]
    return transformer_most_similar_doc_indices

In [ ]:
def tfidf_search(text, K, expansion=False):
    query = [preprocess_text(text)]
    tfidf_query_vec = tfidf_vectorizer.transform(query)
    tfidf_similarities = cosine_similarity(tfidf_embeddings, tfidf_query_vec).flatten()
    if expansion:
        sorted_sim = np.argsort(tfidf_similarities, axis=0)
        most_similar_emb = [tfidf_embeddings[i] for i in sorted_sim[:-11:-1]]
        least_similar_emb = [tfidf_embeddings[i] for i in sorted_sim[:10]]
        expanded = query_expansion_embedding(tfidf_query_vec, most_similar_emb, least_similar_emb)
        tfidf_similarities = cosine_similarity(tfidf_embeddings, expanded).flatten()

    tfidf_most_similar_doc_indices = np.argsort(tfidf_similarities, axis=0)[:-K-1:-1]
    return tfidf_most_similar_doc_indices

In [ ]:
def ft_weighted_search(text, K, expansion):
    changed_text = preprocess_text(text)
    init_emb = np.zeros(embedding_size)
    for word in word_tokenize(changed_text):
        try:
            emb = ft_model.wv[word]
            resault = np.where(features == word)[0]
            if resault.size > 0:
                init_emb += idfs[resault[0]] * emb / np.linalg.norm(emb)
        except:
            pass
    ft_similarities = cosine_similarity(fast_weighted_embeddings, init_emb.reshape(1, -1)).flatten()
    if expansion:
        sorted_sim = np.argsort(ft_similarities, axis=0)
        most_similar_emb = [fast_weighted_embeddings[i] for i in sorted_sim[:-11:-1]]
        least_similar_emb = [fast_weighted_embeddings[i] for i in sorted_sim[:10]]
        expanded = query_expansion_embedding(init_emb, most_similar_emb, least_similar_emb)
        ft_similarities = cosine_similarity(fast_weighted_embeddings, expanded.reshape(1, -1)).flatten()

    ft_most_similar_doc_indices = np.argsort(ft_similarities, axis=0)[:-K-1:-1]
    return ft_most_similar_doc_indices

In [ ]:
def query_search(query_text, output_num, search_type, expansion=False):
    result_indices = []
    if search_type == 'boolean':
        if expansion:
            pass
        result_indices = boolean_search(query_text, output_num)
    elif search_type == 'transformer':
        result_indices = transformer_search(query_text, output_num, expansion)
    elif search_type == 'tfidf':
        result_indices = tfidf_search(query_text, output_num, expansion)
    elif search_type == 'fasttext':
        result_indices = ft_weighted_search(query_text, output_num, expansion)
    
    result = [news_details[i] for i in result_indices]
    return result

In [ ]:
query = 'تیم فوتبال پرسپولیس'
print([i['url'] for i in query_search(query, 10, 'fasttext', True)])
print([i['url'] for i in query_search(query, 10, 'fasttext', False)])

['https://www.yjc.news/fa/news/8177512/تاریخ-مسابقات-لیگ-برتر-و-اردوی-تیم-ملی-فوتبال-اعلام-شد', 'https://www.yjc.news/fa/news/8162628/سمیعی-باشگاه-پانتولیکوس-یونان-از-استقلال-شکایت-کرده-است#comments', 'https://www.yjc.news/fa/news/8170441/رکورد-شکنی-فرزانه-فصیحی-در-ماده-۱۰۰-متر-بانوان', 'https://www.yjc.news/fa/news/8177591/نمایندگان-اسکیت-ایران-در-مسابقات-کاپ-آزاد-ایتالیا-طلایی-شدند', 'https://www.yjc.news/fa/news/8172584/لیگ-ملت\u200cهای-والیبال-۲۰۲۲-ایتالیا-میزبان-مرحله-پایانی-شد', 'http://www.yjc.news/fa/news/8175327/صلاح-در-لیورپول-ماندنی-شد&via=yjcagency', 'https://www.yjc.news/fa/news/8171357/نتایج-دوندگان-کشورمان-در-مسابقات-دو-و-میدانی-جام-کازانوف', 'https://www.yjc.news/fa/news/8172660/برگزاری-فینال-لیگ-دسته-یک-کشتی-همزمان-با-تولد-آقا-تختی', 'https://www.yjc.news/fa/news/8188227/شکسته-شدن-۳-رکورد-ملی-در-رقابت\u200cهای-دو-و-میدانی-قهرمانی-باشگاه\u200cهای-کشور', 'https://www.yjc.news/fa/news/8172592/اعزام-اسنوکرباز\u200cهای-جوان-ایران-به-مسابقات-جهانی']
['https://www.yjc.news/fa